# Basic Weather Agent with MCP Support (Stdio Transport)

* Collaborator: Roberto Rodriguez @Cyb3rWard0g

## Install Required Libraries
Before starting, ensure the required libraries are installed:

In [ ]:
!pip install dapr-agents python-dotenv starlette

## Import Environment Variables

In [1]:
from dotenv import load_dotenv
load_dotenv()  # take environment variables from .env.

True

## Enable Logging

In [2]:
import logging

logging.basicConfig(level=logging.INFO)

## Connect to One MCP Server

In [ ]:
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

from dapr_agents import Agent, AgentTool

# Define the server parameters for the MCP server
server_params = StdioServerParameters(
    command="python",
    args=["servers/weather_server.py", "--server_type", "stdio"],
)

# Connect to the server using stdio
async with stdio_client(server_params) as (read, write):
    async with ClientSession(read, write) as session:
        # Initialize the session
        await session.initialize()
        
        # Get the agent tools from the MCP session
        agent_tools = await AgentTool.from_mcp_session(session)
        
        # Create an agent with the tools
        agent = Agent(
            name="Rob",
            role="Weather Assistant",
            tools=agent_tools
        )

        # Run the agent with a sample query
        await agent.run("What is the weather in Virginia?")

/Users/wardog/Documents/GitHub/dapr-agents/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:dapr_agents.llm.openai.client.base:Initializing OpenAI client...
INFO:dapr_agents.tool.executor:Tool registered: GetWeather
INFO:dapr_agents.tool.executor:Tool registered: GetHumidity
INFO:dapr_agents.tool.executor:Tool Executor initialized with 2 tool(s).
INFO:dapr_agents.agents.base:Constructing system_prompt from agent attributes.
INFO:dapr_agents.agents.base:Using system_prompt to create the prompt template.
INFO:dapr_agents.agents.base:Pre-filled prompt template with attributes: ['name', 'role', 'goal']
INFO:dapr_agents.agents.agent.agent:Iteration 1/10 started.
INFO:dapr_agents.llm.utils.request:Tools are available in the request.
INFO:dapr_agents.llm.openai.chat:Invoking ChatCompletion API.


user:
What is the weather in Virginia?

--------------------------------------------------------------------------------



INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:dapr_agents.llm.openai.chat:Chat completion retrieved successfully.
INFO:dapr_agents.agents.agent.agent:Executing GetWeather with arguments {'location': 'Virginia'}
INFO:dapr_agents.tool.executor:Running tool (auto): GetWeather
INFO:dapr_agents.agents.agent.agent:Iteration 2/10 started.
INFO:dapr_agents.llm.utils.request:Tools are available in the request.
INFO:dapr_agents.llm.openai.chat:Invoking ChatCompletion API.


assistant:
{'content': None, 'role': 'assistant', 'tool_calls': [{'id': 'call_hAs5sH0M102p8cevy8Futc9c', 'type': 'function', 'function': {'name': 'GetWeather', 'arguments': '{"location":"Virginia"}'}}]}

--------------------------------------------------------------------------------

GetWeather(tool) (Id: call_hAs5sH0M102p8cevy8Futc9c):
Virginia: 62F.

--------------------------------------------------------------------------------



INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:dapr_agents.llm.openai.chat:Chat completion retrieved successfully.


assistant:
{'content': 'The current weather in Virginia is 62°F.', 'role': 'assistant'}

--------------------------------------------------------------------------------



## Connect to Multiple MCP Servers

In [ ]:
# Initialize MCP SSE Server
# uv run servers/math_server.py --server_type sse

In [ ]:
# Define your MCP server configs
mcp_servers = {
    "weather": {
        "transport": "stdio",
        "command": "python",
        "args": ["servers/weather_server.py", "--server_type", "stdio"]
    },
    "math": {
        "transport": "sse",
        "url": "http://localhost:8000/sse"
    }
}

In [8]:
# Note: The server script (serverA.py) should be running for this to work.
from dapr_agents.tool.mcp.client import MCPClient

# Create an instance of the MCPClient to interact with the MCP servers
#client = MCPClient(persistent_connections=True)
client = MCPClient()

In [9]:
# Connect to the MCP servers and get tools and prompts
await client.connect_from_config(mcp_servers)

INFO:dapr_agents.tool.mcp.client:Loaded 2 tools from server 'weather'
INFO:dapr_agents.tool.mcp.client:Loaded 0 prompts from server 'weather': 
INFO:dapr_agents.tool.mcp.client:Successfully connected to MCP server 'weather' (ephemeral mode)
INFO:mcp.client.sse:Connecting to SSE endpoint: http://localhost:8000/sse
INFO:httpx:HTTP Request: GET http://localhost:8000/sse "HTTP/1.1 200 OK"
INFO:mcp.client.sse:Received endpoint URL: http://localhost:8000/messages/?session_id=712265480ddd4b3b9d05ac68f2c04f81
INFO:mcp.client.sse:Starting post writer with endpoint URL: http://localhost:8000/messages/?session_id=712265480ddd4b3b9d05ac68f2c04f81
INFO:httpx:HTTP Request: POST http://localhost:8000/messages/?session_id=712265480ddd4b3b9d05ac68f2c04f81 "HTTP/1.1 202 Accepted"
INFO:httpx:HTTP Request: POST http://localhost:8000/messages/?session_id=712265480ddd4b3b9d05ac68f2c04f81 "HTTP/1.1 202 Accepted"
INFO:httpx:HTTP Request: POST http://localhost:8000/messages/?session_id=712265480ddd4b3b9d05ac68

In [10]:
# Get all tools available
tools = client.get_all_tools()
print("🔧 Tools:", [t.name for t in tools])

🔧 Tools: ['WeatherGetWeather', 'WeatherGetHumidity', 'MathAdd', 'MathMultiply']


## Initialize Agent

In [11]:
from dapr_agents import Agent

agent = Agent(
    name="Rob",
    role= "Weather Assistant",
    tools=tools
)

INFO:dapr_agents.llm.openai.client.base:Initializing OpenAI client...
INFO:dapr_agents.tool.executor:Tool registered: WeatherGetWeather
INFO:dapr_agents.tool.executor:Tool registered: WeatherGetHumidity
INFO:dapr_agents.tool.executor:Tool registered: MathAdd
INFO:dapr_agents.tool.executor:Tool registered: MathMultiply
INFO:dapr_agents.tool.executor:Tool Executor initialized with 4 tool(s).
INFO:dapr_agents.agents.base:Constructing system_prompt from agent attributes.
INFO:dapr_agents.agents.base:Using system_prompt to create the prompt template.
INFO:dapr_agents.agents.base:Pre-filled prompt template with attributes: ['name', 'role', 'goal']


## Run Agent

In [12]:
#await agent.run("What is the weather in New York?")

await agent.run("What is 2 plus 2?")

INFO:dapr_agents.agents.agent.agent:Iteration 1/10 started.
INFO:dapr_agents.llm.utils.request:Tools are available in the request.
INFO:dapr_agents.llm.openai.chat:Invoking ChatCompletion API.


user:
What is 2 plus 2?

--------------------------------------------------------------------------------



INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:dapr_agents.llm.openai.chat:Chat completion retrieved successfully.
INFO:dapr_agents.agents.agent.agent:Executing MathAdd with arguments {'a': 2, 'b': 2}
INFO:dapr_agents.tool.executor:Running tool (auto): MathAdd
INFO:mcp.client.sse:Connecting to SSE endpoint: http://localhost:8000/sse
INFO:httpx:HTTP Request: GET http://localhost:8000/sse "HTTP/1.1 200 OK"
INFO:mcp.client.sse:Received endpoint URL: http://localhost:8000/messages/?session_id=6eef0e0574be4dfd9feabc16d32ec48f
INFO:mcp.client.sse:Starting post writer with endpoint URL: http://localhost:8000/messages/?session_id=6eef0e0574be4dfd9feabc16d32ec48f
INFO:httpx:HTTP Request: POST http://localhost:8000/messages/?session_id=6eef0e0574be4dfd9feabc16d32ec48f "HTTP/1.1 202 Accepted"
INFO:httpx:HTTP Request: POST http://localhost:8000/messages/?session_id=6eef0e0574be4dfd9feabc16d32ec48f "HTTP/1.1 202 Accepted"
INFO:httpx:HTTP Request: POS

assistant:
{'content': None, 'role': 'assistant', 'tool_calls': [{'id': 'call_MONcDw2a2zZZIGr8OJowp2Ec', 'type': 'function', 'function': {'name': 'MathAdd', 'arguments': '{"a":2,"b":2}'}}]}

--------------------------------------------------------------------------------

MathAdd(tool) (Id: call_MONcDw2a2zZZIGr8OJowp2Ec):
4

--------------------------------------------------------------------------------



INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:dapr_agents.llm.openai.chat:Chat completion retrieved successfully.


assistant:
{'content': '2 plus 2 equals 4.', 'role': 'assistant'}

--------------------------------------------------------------------------------



'2 plus 2 equals 4.'

In [13]:
await agent.run("What is the weather in NY?")

INFO:dapr_agents.agents.agent.agent:Iteration 1/10 started.
INFO:dapr_agents.llm.utils.request:Tools are available in the request.
INFO:dapr_agents.llm.openai.chat:Invoking ChatCompletion API.


user:
What is the weather in NY?

--------------------------------------------------------------------------------



INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:dapr_agents.llm.openai.chat:Chat completion retrieved successfully.
INFO:dapr_agents.agents.agent.agent:Executing WeatherGetWeather with arguments {'location': 'New York'}
INFO:dapr_agents.tool.executor:Running tool (auto): WeatherGetWeather


assistant:
{'content': None, 'role': 'assistant', 'tool_calls': [{'id': 'call_liUykrnpgqfLzTZh3Ny5mOr4', 'type': 'function', 'function': {'name': 'WeatherGetWeather', 'arguments': '{"location":"New York"}'}}]}

--------------------------------------------------------------------------------



INFO:dapr_agents.agents.agent.agent:Iteration 2/10 started.
INFO:dapr_agents.llm.utils.request:Tools are available in the request.
INFO:dapr_agents.llm.openai.chat:Invoking ChatCompletion API.


WeatherGetWeather(tool) (Id: call_liUykrnpgqfLzTZh3Ny5mOr4):
New York: 76F.

--------------------------------------------------------------------------------



INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:dapr_agents.llm.openai.chat:Chat completion retrieved successfully.


assistant:
{'content': 'The current weather in New York is 76°F.', 'role': 'assistant'}

--------------------------------------------------------------------------------



'The current weather in New York is 76°F.'

In [14]:
await agent.run("What is 2 times 2?")

INFO:dapr_agents.agents.agent.agent:Iteration 1/10 started.
INFO:dapr_agents.llm.utils.request:Tools are available in the request.
INFO:dapr_agents.llm.openai.chat:Invoking ChatCompletion API.


user:
What is 2 times 2?

--------------------------------------------------------------------------------



INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:dapr_agents.llm.openai.chat:Chat completion retrieved successfully.
INFO:dapr_agents.agents.agent.agent:Executing MathMultiply with arguments {'a': 2, 'b': 2}
INFO:dapr_agents.tool.executor:Running tool (auto): MathMultiply
INFO:mcp.client.sse:Connecting to SSE endpoint: http://localhost:8000/sse
INFO:httpx:HTTP Request: GET http://localhost:8000/sse "HTTP/1.1 200 OK"
INFO:mcp.client.sse:Received endpoint URL: http://localhost:8000/messages/?session_id=69b6a4e42efb42b6a8e57236054f8dc2
INFO:mcp.client.sse:Starting post writer with endpoint URL: http://localhost:8000/messages/?session_id=69b6a4e42efb42b6a8e57236054f8dc2
INFO:httpx:HTTP Request: POST http://localhost:8000/messages/?session_id=69b6a4e42efb42b6a8e57236054f8dc2 "HTTP/1.1 202 Accepted"
INFO:httpx:HTTP Request: POST http://localhost:8000/messages/?session_id=69b6a4e42efb42b6a8e57236054f8dc2 "HTTP/1.1 202 Accepted"
INFO:httpx:HTTP Re

assistant:
{'content': None, 'role': 'assistant', 'tool_calls': [{'id': 'call_gSzbDFop9IHELtKkbPFYCCur', 'type': 'function', 'function': {'name': 'MathMultiply', 'arguments': '{"a":2,"b":2}'}}]}

--------------------------------------------------------------------------------

MathMultiply(tool) (Id: call_gSzbDFop9IHELtKkbPFYCCur):
4

--------------------------------------------------------------------------------



INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:dapr_agents.llm.openai.chat:Chat completion retrieved successfully.


assistant:
{'content': '2 times 2 equals 4.', 'role': 'assistant'}

--------------------------------------------------------------------------------



'2 times 2 equals 4.'

In [15]:
await agent.run("What is 2 times 3?")

INFO:dapr_agents.agents.agent.agent:Iteration 1/10 started.
INFO:dapr_agents.llm.utils.request:Tools are available in the request.
INFO:dapr_agents.llm.openai.chat:Invoking ChatCompletion API.


user:
What is 2 times 3?

--------------------------------------------------------------------------------



INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:dapr_agents.llm.openai.chat:Chat completion retrieved successfully.
INFO:dapr_agents.agents.agent.agent:Executing MathMultiply with arguments {'a': 2, 'b': 3}
INFO:dapr_agents.tool.executor:Running tool (auto): MathMultiply
INFO:mcp.client.sse:Connecting to SSE endpoint: http://localhost:8000/sse
INFO:httpx:HTTP Request: GET http://localhost:8000/sse "HTTP/1.1 200 OK"
INFO:mcp.client.sse:Received endpoint URL: http://localhost:8000/messages/?session_id=19f3b1164f144b668e1f1cacaffff2a0
INFO:mcp.client.sse:Starting post writer with endpoint URL: http://localhost:8000/messages/?session_id=19f3b1164f144b668e1f1cacaffff2a0
INFO:httpx:HTTP Request: POST http://localhost:8000/messages/?session_id=19f3b1164f144b668e1f1cacaffff2a0 "HTTP/1.1 202 Accepted"
INFO:httpx:HTTP Request: POST http://localhost:8000/messages/?session_id=19f3b1164f144b668e1f1cacaffff2a0 "HTTP/1.1 202 Accepted"
INFO:httpx:HTTP Re

assistant:
{'content': None, 'role': 'assistant', 'tool_calls': [{'id': 'call_yLVMJfOPjW5EpwY9ncxVyVGR', 'type': 'function', 'function': {'name': 'MathMultiply', 'arguments': '{"a":2,"b":3}'}}]}

--------------------------------------------------------------------------------

MathMultiply(tool) (Id: call_yLVMJfOPjW5EpwY9ncxVyVGR):
6

--------------------------------------------------------------------------------



INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:dapr_agents.llm.openai.chat:Chat completion retrieved successfully.


assistant:
{'content': '2 times 3 equals 6.', 'role': 'assistant'}

--------------------------------------------------------------------------------



'2 times 3 equals 6.'